In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np

In [4]:
import os
import tensorflow
from keras.preprocessing import image

In [5]:
import pandas as pd

In [6]:
categories =['with_mask', 'without_mask']


In [7]:
data=[]
for category in categories:
    path=os.path.join('train', category)
    label=categories.index(category)
    for file in os.listdir(path):
        img_path=os.path.join(path, file)
        img=cv2.imread(img_path)
        img=cv2.resize(img, (224, 224))
#         print(img.shape)
        data.append([img, label])
        

In [8]:
len(data)

897

In [9]:
import random
random.shuffle(data)

In [10]:
x=[]
y=[]
for features, labels in data:
    x.append(features)
    y.append(labels)

In [11]:
x=np.array(x)
y=np.array(y)

In [12]:
x.shape

(897, 224, 224, 3)

In [13]:
x=x/255

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2)

In [15]:
x_train.shape

(717, 224, 224, 3)

In [16]:
x_test.shape

(180, 224, 224, 3)

In [17]:
from keras.applications.vgg16 import VGG16

In [18]:
vgg=VGG16()

In [19]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [20]:
from keras import Sequential

In [21]:
model= Sequential()

In [22]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [24]:
for layer in model.layers:
    layer.trainable=False

In [25]:
from keras.layers import Dense

In [26]:
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [28]:
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"])

In [29]:
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
23/23 [==============================] - 76s 3s/step - loss: 0.6348 - accuracy: 0.6276 - val_loss: 0.4699 - val_accuracy: 0.8389
Epoch 2/5
23/23 [==============================] - 85s 4s/step - loss: 0.4171 - accuracy: 0.8466 - val_loss: 0.3364 - val_accuracy: 0.8944
Epoch 3/5
23/23 [==============================] - 85s 4s/step - loss: 0.3103 - accuracy: 0.9135 - val_loss: 0.2492 - val_accuracy: 0.9444
Epoch 4/5
23/23 [==============================] - 91s 4s/step - loss: 0.2576 - accuracy: 0.9247 - val_loss: 0.2129 - val_accuracy: 0.9500
Epoch 5/5
23/23 [==============================] - 90s 4s/step - loss: 0.2314 - accuracy: 0.9344 - val_loss: 0.1904 - val_accuracy: 0.9444


In [50]:
cap=cv2.VideoCapture(0)

In [75]:
def detect_face_mask(img):
    y_pred=model.predict(img.reshape(1,224,224,3))
    if(y_pred[0][0]>=0.5):
        return 1
    else:
        return 0
#     return y_pred[0][0]

In [77]:
while True:
    
    ret, frame=cap.read()
    img=cv2.resize(frame, (224,224))
    y_pred=detect_face_mask(img)
    if y_pred ==0:
        draw_label(frame, "Mask", (30, 30), (0, 255, 0))
    else:
        draw_label(frame, "NO Mask", (30, 30), (255, 0, 0))
    cv2.imshow("window", frame)
    if cv2.waitKey(1) & 0XFF == ord('x'):
        break;
cv2.destroyAllWindows()

1/1 [==============================] - 0s 138ms/step


1/1 [==============================] - 0s 141ms/step


1/1 [==============================] - 0s 145ms/step


In [71]:
def draw_label(img, text, pos, bg_color):
    text_size=cv2.getTextSize(text, cv2.FONT_HERSHEY_COMPLEX, 1, cv2.FILLED)
    end_x = pos[0]+text_size[0][0]+2
    end_y = pos[1]+text_size[0][1]-2
    cv2.rectangle(img ,pos, (end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 1)
    